# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Apr 21 2023 8:35AM,261548,22,646479,"NBTY Global, Inc.",Released
1,Apr 21 2023 8:33AM,261547,16,9398934,Sartorius Bioprocess Solutions,Released
2,Apr 21 2023 8:30AM,261546,20,9396795,"Exact-Rx, Inc.",Released
3,Apr 21 2023 8:30AM,261546,20,9396796,"Exact-Rx, Inc.",Released
4,Apr 21 2023 8:30AM,261546,20,9397835,"Exact-Rx, Inc.",Released
5,Apr 21 2023 8:29AM,261545,16,SHL-9398950,"SHL Pharma, LLC",Released
6,Apr 21 2023 8:22AM,261543,18,EMERSA-9401023,"Emersa Waterbox, LLC",Released
7,Apr 21 2023 8:21AM,261544,22,646478,"NBTY Global, Inc.",Released
8,Apr 21 2023 8:01AM,261542,19,9400522,"Graystone, LLC",Released
9,Apr 20 2023 4:17PM,261534,19,TR-RG-04242023,"GCH Granules USA, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
7,261544,Released,1
8,261545,Released,1
9,261546,Released,3
10,261547,Released,1
11,261548,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
261544,NaN,1.0
261545,NaN,1.0
261546,NaN,3.0
261547,NaN,1.0
261548,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
261484,0.0,1.0
261505,1.0,0.0
261514,0.0,2.0
261520,0.0,1.0
261534,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
261484,0,1
261505,1,0
261514,0,2
261520,0,1
261534,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,261484,0,1
1,261505,1,0
2,261514,0,2
3,261520,0,1
4,261534,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,261484,,1
1,261505,1,
2,261514,,2
3,261520,,1
4,261534,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Apr 21 2023 8:35AM,261548,22,"NBTY Global, Inc."
1,Apr 21 2023 8:33AM,261547,16,Sartorius Bioprocess Solutions
2,Apr 21 2023 8:30AM,261546,20,"Exact-Rx, Inc."
5,Apr 21 2023 8:29AM,261545,16,"SHL Pharma, LLC"
6,Apr 21 2023 8:22AM,261543,18,"Emersa Waterbox, LLC"
7,Apr 21 2023 8:21AM,261544,22,"NBTY Global, Inc."
8,Apr 21 2023 8:01AM,261542,19,"Graystone, LLC"
9,Apr 20 2023 4:17PM,261534,19,"GCH Granules USA, Inc."
10,Apr 20 2023 1:50PM,261520,10,Yusen – 3D Matrix
11,Apr 20 2023 1:13PM,261514,15,"Alliance Pharma, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Apr 21 2023 8:35AM,261548,22,"NBTY Global, Inc.",,1
1,Apr 21 2023 8:33AM,261547,16,Sartorius Bioprocess Solutions,,1
2,Apr 21 2023 8:30AM,261546,20,"Exact-Rx, Inc.",,3
3,Apr 21 2023 8:29AM,261545,16,"SHL Pharma, LLC",,1
4,Apr 21 2023 8:22AM,261543,18,"Emersa Waterbox, LLC",,1
5,Apr 21 2023 8:21AM,261544,22,"NBTY Global, Inc.",,1
6,Apr 21 2023 8:01AM,261542,19,"Graystone, LLC",,1
7,Apr 20 2023 4:17PM,261534,19,"GCH Granules USA, Inc.",,1
8,Apr 20 2023 1:50PM,261520,10,Yusen – 3D Matrix,,1
9,Apr 20 2023 1:13PM,261514,15,"Alliance Pharma, Inc.",,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Apr 21 2023 8:35AM,261548,22,"NBTY Global, Inc.",1,
1,Apr 21 2023 8:33AM,261547,16,Sartorius Bioprocess Solutions,1,
2,Apr 21 2023 8:30AM,261546,20,"Exact-Rx, Inc.",3,
3,Apr 21 2023 8:29AM,261545,16,"SHL Pharma, LLC",1,
4,Apr 21 2023 8:22AM,261543,18,"Emersa Waterbox, LLC",1,
5,Apr 21 2023 8:21AM,261544,22,"NBTY Global, Inc.",1,
6,Apr 21 2023 8:01AM,261542,19,"Graystone, LLC",1,
7,Apr 20 2023 4:17PM,261534,19,"GCH Granules USA, Inc.",1,
8,Apr 20 2023 1:50PM,261520,10,Yusen – 3D Matrix,1,
9,Apr 20 2023 1:13PM,261514,15,"Alliance Pharma, Inc.",2,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Apr 21 2023 8:35AM,261548,22,"NBTY Global, Inc.",1,
1,Apr 21 2023 8:33AM,261547,16,Sartorius Bioprocess Solutions,1,
2,Apr 21 2023 8:30AM,261546,20,"Exact-Rx, Inc.",3,
3,Apr 21 2023 8:29AM,261545,16,"SHL Pharma, LLC",1,
4,Apr 21 2023 8:22AM,261543,18,"Emersa Waterbox, LLC",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Apr 21 2023 8:35AM,261548,22,"NBTY Global, Inc.",1.0,NaN
1,Apr 21 2023 8:33AM,261547,16,Sartorius Bioprocess Solutions,1.0,NaN
2,Apr 21 2023 8:30AM,261546,20,"Exact-Rx, Inc.",3.0,NaN
3,Apr 21 2023 8:29AM,261545,16,"SHL Pharma, LLC",1.0,NaN
4,Apr 21 2023 8:22AM,261543,18,"Emersa Waterbox, LLC",1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Apr 21 2023 8:35AM,261548,22,"NBTY Global, Inc.",1.0,0.0
1,Apr 21 2023 8:33AM,261547,16,Sartorius Bioprocess Solutions,1.0,0.0
2,Apr 21 2023 8:30AM,261546,20,"Exact-Rx, Inc.",3.0,0.0
3,Apr 21 2023 8:29AM,261545,16,"SHL Pharma, LLC",1.0,0.0
4,Apr 21 2023 8:22AM,261543,18,"Emersa Waterbox, LLC",1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,261520,1.0,0.0
15,261514,2.0,0.0
16,523092,2.0,0.0
18,261543,1.0,0.0
19,784560,3.0,0.0
20,261546,3.0,0.0
21,261505,0.0,1.0
22,523092,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,261520,1.0,0.0
1,15,261514,2.0,0.0
2,16,523092,2.0,0.0
3,18,261543,1.0,0.0
4,19,784560,3.0,0.0
5,20,261546,3.0,0.0
6,21,261505,0.0,1.0
7,22,523092,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,1.0,0.0
1,15,2.0,0.0
2,16,2.0,0.0
3,18,1.0,0.0
4,19,3.0,0.0
5,20,3.0,0.0
6,21,0.0,1.0
7,22,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,1.0
1,15,Released,2.0
2,16,Released,2.0
3,18,Released,1.0
4,19,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,20,21,22
Status,,,,,,,,
Executing,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
Released,1.0,2.0,2.0,1.0,3.0,3.0,0.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,20,21,22
0,Executing,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,Released,1.0,2.0,2.0,1.0,3.0,3.0,0.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,20,21,22
0,Executing,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,Released,1.0,2.0,2.0,1.0,3.0,3.0,0.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()